## Servicio Social – Comparativa de Modelos de Visión Computacional para Detección de Contaminantes en Cosechas de Chile 🌶️

In [2]:
import torch

print(torch.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # esto por si hay disponible una GPU si no usa la CPU
print("CUDA:", torch.cuda.is_available())

2.9.0+cu126
CUDA: True


In [3]:
import zipfile
import os

zip_file_path = '/content/contaminante.zip'
extract_dir = '/content/'

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"'{zip_file_path}' decompressed to '{extract_dir}'")

'/content/contaminante.zip' decompressed to '/content/'


## Integridad de los datos

In [4]:
import os

base_path = r"/content/contaminante"
data_types = ["train", "valid", "test"]

for split in data_types:
    images_path = os.path.join(base_path, split, "images")
    labels_path = os.path.join(base_path, split, "labels")

    num_images = len([
        f for f in os.listdir(images_path)
        if f.lower().endswith((".png", ".jpg", ".jpeg"))
    ])

    num_labels = len(os.listdir(labels_path))

    print(f"{split.upper()}:")
    print(f"  Imágenes: {num_images}")
    print(f"  Labels:   {num_labels}")


TRAIN:
  Imágenes: 3704
  Labels:   3704
VALID:
  Imágenes: 1236
  Labels:   1236
TEST:
  Imágenes: 1232
  Labels:   1232


### Pre-Procesamiento de las imagenes 

### Implementacion del modelo: YoloV8

In [ ]:
# =========================
# 1. IMPORTACIONES
# =========================
from ultralytics import YOLO
import os

# =========================
# 2. VERIFICAR RUTAS (opcional pero recomendado)
# =========================
assert os.path.exists("data.yaml"), "No se encontró data.yaml"
assert os.path.exists("dataset/train/images"), "No existe train/images"
assert os.path.exists("dataset/train/labels"), "No existe train/labels"

# =========================
# 3. INSTANCIAR EL MODELO
# =========================
# Aquí NO entrenas nada todavía
# Solo cargas YOLO con pesos preentrenados
model = YOLO("yolov8n.pt")

# =========================
# 4. ENTRENAMIENTO
# =========================
# AQUÍ es donde YOLO:
# - Lee data.yaml
# - Carga imágenes
# - Carga etiquetas
# - Ajusta el modelo a tu problema
model.train(
    data="data.yaml",   # Archivo que tú creaste
    epochs=50,          # Número de épocas
    imgsz=640,          # Tamaño de imagen
    batch=16,           # Tamaño de lote
    device="cuda"        # Quita esto si usas GPU
)

# =========================
# 5. EVALUACIÓN (VALIDACIÓN)
# =========================
# Evalúa el modelo entrenado con el set de validación
metrics = model.val()

print(metrics)

# =========================
# 6. CARGAR EL MEJOR MODELO ENTRENADO
# =========================
# YOLO guarda los pesos automáticamente
best_model_path = "runs/detect/train/weights/best.pt"

model = YOLO(best_model_path)

# =========================
# 7. INFERENCIA (DETECCIÓN)
# =========================
# Probar el modelo con imágenes nuevas
results = model.predict(
    source="dataset/test/images",  # Carpeta o imagen
    conf=0.25,
    save=True
)

print("Inferencia completada")


### Evaluacion del modelo